Question 1: 
Write a pseudo-code for how you would implement this with a set of nested
for loops. The convolution is defined by a set of weights/parameters which we will learn.
How do you represent these weights?


In [ ]:
for sample in batch_size:
    padding_width = input_width + 2*padding
    padding_height = input_height + 2*padding
    # Given that output_width and output_height are the same:
    output_width = output_height = ((input_width - kernel_size + 2*padding)/stride) + 1
    # Create empty output for this layer
    output = zeros([total_kernels, output_width, output_height])
    for layer in input_channels:
        layer_pad = [sample, layer, :, :]
        # Add padding
        for pad in padding:
            add column of zeros to right to layer_pad
            add column of zeros to left to layer_pad
            add column of zeros to top to layer_pad
            add column of zeros to bottom to layer_pad
        # Move over layer:
        for y in range(output_height-1):
            for x in range(output_width-1):
                # If multiple kernels exist, save in different slices of output
                for kernel in total_kernels:
                    patch = layer_pad[y*stride : (y*stride)+kernel_size, x*stride : (x*stride)+kernel_size]
                    output[kernel, y, x] += sum( patch * kernel[layer, :, :] )
    sample_output[sample, :, :, :] = output


Question 2. 
For a given input tensor, kernel size, stride and padding (no dilutions) work out
a general function that computes the size of the output.
https://pytorch.org/docs/stable/generated/torch.nn.functional.conv2d.html?highlight=functional%20conv2d#torch.nn.functional.conv2d 

In [3]:
import torch
import numpy as np

In [13]:
tensor = np.random.rand(1, 2, 27, 27)
weight = np.random.rand(1, 2, 3, 3)

def output_size(input_tensor, kernel_size, stride, padding):
    input_size = len(input_tensor[0,0,:])
    out_size = (((input_size - kernel_size + 2*padding) / stride ) + 1)
    return out_size

print(output_size(tensor, 3, 2, 0)) #<- example from the slides: Lecture 3 AlexNet, top right

tensor = torch.rand([1, 2, 27, 27])
weight = torch.rand([1, 2, 3, 3])

print(torch.nn.functional.conv2d(tensor, weight, stride=2,padding=0).size())


13.0
torch.Size([1, 1, 13, 13])


Question 3: 
Write a naive (non-vectorized) implementation of the unfold function in
pseudocode. Include the pseudocode in your report.
https://pytorch.org/docs/stable/generated/torch.nn.functional.unfold.html 

In [ ]:
#input_tensor = [b, c, h, w] #batch_size, channels, height, width

# Pseudo code naive unfold:
def naive_unfold(input_tensor, kernel_size, stride, padding):
    channels = len(input_tensor[0, :])
    width = len(input_tensor[0, 0, :])
    padded_width = width + 2*padding
    k = kernel_size * kernel_size * channels

    output_size = output_size(input_tensor, kernel_size, stride, padding)

    #1 extract all patches from the input
    for sample in b:
        for channel in c:
            layer_pad = [sample, layer, :, :]
            # Add padding
            for pad in padding:
                add column of zeros to right to layer_pad
                add column of zeros to left to layer_pad
                add column of zeros to top to layer_pad
                add column of zeros to bottom to layer_pad
            n_patches_per_layer = output_size * output_size
            for y in range(output_size-1):
                for x in range(output_size-1):
                    # x+ y = number of patch, total_patch is 0 first time
                    total_patch = (x+y) * channel
                    patch[sample, x+y + total_patch ,:, :] = layer_pad[y*stride : (y*stride)+kernel_size, 
                                                                        x*stride : (x*stride)+kernel_size]
    # 2. Flatten these patches (with all channels) into vectors, arranged as the columns of a matrix X.
    #### THIS CORRECT???
    X = patch[:, :].flatten()
    p = len(X)
    # 3. Multiply this matrix by a weight matrix Y = XW
    Y = X * W
    # 4. Reshape the matrix Y, so that its columns become the pixels of the output tensor.
    k = c * output_size * output_size
    Y = Y.reshape([b, k, p])
    return patch
    #output = [b, k, p] #batch_size, number of values per patch, number of patches

In [17]:
import torch
import torch.nn.functional as F
from torch import nn

class Conv2D(nn.Module):
    def __init__(self,in_channels, out_channels, kernel_size=(3,3), stride=1, padding=1):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
    def forward(self, input_batch):

        b, c, h, w = input_batch.size()
        # your code here
        unfold = F.unfold(input_batch, self.kernel_size, padding=self.padding, stride=self.stride)
        print(unfold.size())
        return unfold

# We use the Conv2D module by instantiating it, and applying it to an input.
conv = Conv2D(3, 1)
input_batch = torch.randn(16, 3, 32, 32)
output_batch = conv(input_batch)

AttributeError: 'Conv2D' object has no attribute '_backward_hooks'